# RAG Post Retrieval Optimization - Reranking featuring Bedrock and llamaindex

Reranking is a crucial post-retrieval process that refines the initial set of retrieved documents to improve the relevance and quality of information used for generating responses. After the initial retrieval step, which typically uses methods like vector similarity search, reranking takes the retrieved documents and re-scores their relevance to the query using more sophisticated techniques. These may include cross-encoders, BERT-based models, or other advanced algorithms that can better capture semantic nuances and context. Based on these scores, the documents are reordered, pushing the most relevant and informative ones to the top, thereby enhancing the accuracy and contextual appropriateness of the final response.

In this lab, we will highlight the usage of Bedrock Reranker LLMs (for example Cohere Rerank 3.5) as a reranker model to improve the relevance of the documents retrieved from Amazon SEC filings. We also utilized the Bedrock Nova Pro model and the Bedrock Titan Text Embedding v2.0 model for natural language processing and document embeddings to build this RAG system. The key steps were: ingesting PDF documents into a vector database index, retrieving top relevant nodes based on a query using semantic search, reranking the retrieved nodes using the fine-tuned transformer model deployed on SageMaker, and synthesizing a final response consolidating the reranked information. See below image for details.

- Vector Database (Faiss / local on the notebook for this demo)
- LLM (Amazon Bedrock - Nova Pro)
- Embeddings Model (Bedrock Titan Text Embedding v2.0)
- ReRanking Model (Cohere Reranker 3.5) running on Bedrock
- Llamaindex for orchestration (ingestion, reranking, retrieval and final response synthesis)
- Datasets (Amazon SEC 10-k statements for year 2022 and 2023 )

## Pre-req
You must run the `[workshop_setup.ipynb]`(../lab00-setup/workshop_setup.ipynb) notebook in `lab00-setup` before starting this lab.

In [1]:
import warnings
warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook") 

/tmp/ipykernel_23283/952814476.py:2: UserWarning: Warning: if you did not run lab00-setup, please go back and run the lab00 notebook
  warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook")



### > Setup
We start by importing necessary llamaindex libraries

In [2]:
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core.postprocessor import LLMRerank
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core import Settings

We select Anthropic Claude3 Sonnet as our LLM. For embedding model, we are selecting Amazon Titan Text Embed v2.0. Chunk size is set at 512 for this example.

In [3]:
import json
from typing import Sequence, List
from llama_index.core.settings import Settings
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.embeddings.bedrock import BedrockEmbedding, Models
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
#from llama_index.core.postprocessor.bedrock_rerank import AWSBedrockRerank

profile_name = "default"

# define the LLM
llm = BedrockConverse(
    model="us.amazon.nova-pro-v1:0",
    profile_name=profile_name,
)

# define the embedding model
embed_model = BedrockEmbedding(model = "amazon.titan-embed-text-v2:0")

Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512

from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool
import nest_asyncio
nest_asyncio.apply()

### > Document Ingestion
We ingest and index the data stored in data directory. The amazon folder has SEC-10k files from 2022 and 2023.

In [4]:
# load data
amazon_secfiles = SimpleDirectoryReader(input_dir="../data/lab03/amazon/").load_data()

# build index
amazon_index = VectorStoreIndex.from_documents(
    amazon_secfiles,
    use_async=True,
)

In [7]:
query_engine = amazon_index.as_query_engine(similarity_top_k=5)
response = query_engine.query(
    "Describe key business risks for Amazon during covid?",
)

print(response)

Key business risks for Amazon during the COVID-19 pandemic included disruptions from natural or human-caused disasters, extreme weather, geopolitical events, and security issues. Additionally, labor or trade disputes, including restrictive governmental actions impacting Amazon, its customers, and third-party sellers and suppliers in foreign countries, posed significant risks. The pandemic also heightened the challenges related to successfully optimizing and operating Amazon's fulfillment network and data centers, as well as managing and retaining sufficient personnel to meet operational needs. Furthermore, the company faced risks related to its supplier relationships, including limited or single-sources of supply and the lack of long-term arrangements to guarantee availability of merchandise and content.


In [8]:
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='bad0e873-f7ad-4c1c-a8ed-8671b5e3523e', embedding=None, metadata={'page_label': '6', 'file_name': 'amazon_2023.pdf', 'file_path': '/home/sagemaker-user/aws-genai-rag-workshop/lab03-07-advance-rag-post-retrieval/../data/lab03/amazon/amazon_2023.pdf', 'file_type': 'application/pdf', 'file_size': 800598, 'creation_date': '2025-03-12', 'last_modified_date': '2024-02-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0e740494-5573-47f9-8a25-ae63cb76a564', node_type='4', metadata={'page_label': '6', 'file_name': 'amazon_2023.pdf', 'file_path': '/home/sagemaker-user/aws-genai-rag-workshop/lab03-07-advance-rag-post-retrieval/../data/lab03/amazon/amazon_2023.pdf', 'file_type': 'ap

# Using Bedrock Rerank LLMs with Llamaindex. We will try with Cohere Reranker3.5

In [12]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.postprocessor.bedrock_rerank import AWSBedrockRerank

In [13]:
reranker = AWSBedrockRerank(
    top_n=3,
    model_id="cohere.rerank-v3-5:0",
    region_name="us-west-2",
)

In [14]:
query_engine = amazon_index.as_query_engine(
    similarity_top_k=5,
    node_postprocessors=[reranker],
)
response = query_engine.query(
    "Describe key business risks for Amazon during covid?",
)

print(response)

The key business risks for Amazon during the COVID-19 period include intense competition across various sectors such as retail, e-commerce services, web and infrastructure computing services, electronic devices, digital content, advertising, grocery, and transportation and logistics. Competitors may have greater resources, longer histories, more customers, and greater brand recognition, which could lead to better vendor terms, more aggressive pricing, and increased investment in technology and marketing. Additionally, the loss of executive officers or other key employees, challenges in hiring and retaining qualified personnel, and labor union efforts to organize employees pose significant risks. Furthermore, reliance on significant suppliers, including content and technology licensors, and the potential for limited or single-source supply chains, introduce additional risks related to the availability and continuity of merchandise and content.


In [15]:
print(response.source_nodes)

[NodeWithScore(node=TextNode(id_='bad0e873-f7ad-4c1c-a8ed-8671b5e3523e', embedding=None, metadata={'page_label': '6', 'file_name': 'amazon_2023.pdf', 'file_path': '/home/sagemaker-user/aws-genai-rag-workshop/lab03-07-advance-rag-post-retrieval/../data/lab03/amazon/amazon_2023.pdf', 'file_type': 'application/pdf', 'file_size': 800598, 'creation_date': '2025-03-12', 'last_modified_date': '2024-02-02'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='0e740494-5573-47f9-8a25-ae63cb76a564', node_type='4', metadata={'page_label': '6', 'file_name': 'amazon_2023.pdf', 'file_path': '/home/sagemaker-user/aws-genai-rag-workshop/lab03-07-advance-rag-post-retrieval/../data/lab03/amazon/amazon_2023.pdf', 'file_type': 'ap